# Libraries

In [1]:
import numpy as np
import pandas as pd
import os
import glob
import nibabel as nib

# UMAP


## Data Loading and Processing

Load the FC maps data in nii.gz format and extract the subjects identifiers

In [24]:
# Path to the directory containing FC maps
path_dir ='/Users/emmatosato/Documents/PhD/General/Verona/FCmaps_mean/*gz'

# Collects all file paths and sort
files=glob.glob(path_dir)
files.sort()

# Extract Subject Identifiers
sub_id = [sub.replace('/Users/emmatosato/Documents/PhD/General/Verona/FCmaps_mean/', '') for sub in files]
sub_id = [sub.replace('.FDC.nii.gz', '') for sub in sub_id]

Flatten and store each subject’s data

In [25]:
# Empty list to hold flattened brain maps
mapsPCFDC=[]

# Loop over files
for x in range(len(files)):
    print(files[x])
    d=nib.load(files[x]).get_fdata().flatten()
    mapsPCFDC.append(d)

# Saving affine transformation matrix
aff = nib.load(files[0]).affine

# Create dataframe
df=pd.DataFrame(mapsPCFDC)
df=df.T

# Add Subject Identifiers
df_fdc=df.T
df_fdc.insert(loc = 0, column = 'ID', value = sub_id)

/Users/emmatosato/Documents/PhD/General/Verona/FCmaps_mean/002_S_4654.FDC.nii.gz
/Users/emmatosato/Documents/PhD/General/Verona/FCmaps_mean/002_S_6695.FDC.nii.gz
/Users/emmatosato/Documents/PhD/General/Verona/FCmaps_mean/003_S_6264.FDC.nii.gz
/Users/emmatosato/Documents/PhD/General/Verona/FCmaps_mean/003_S_6268.FDC.nii.gz
/Users/emmatosato/Documents/PhD/General/Verona/FCmaps_mean/003_S_6432.FDC.nii.gz
/Users/emmatosato/Documents/PhD/General/Verona/FCmaps_mean/003_S_6479.FDC.nii.gz
/Users/emmatosato/Documents/PhD/General/Verona/FCmaps_mean/003_S_6606.FDC.nii.gz
/Users/emmatosato/Documents/PhD/General/Verona/FCmaps_mean/006_S_6252.FDC.nii.gz
/Users/emmatosato/Documents/PhD/General/Verona/FCmaps_mean/006_S_6682.FDC.nii.gz
/Users/emmatosato/Documents/PhD/General/Verona/FCmaps_mean/007_S_6341.FDC.nii.gz
/Users/emmatosato/Documents/PhD/General/Verona/FCmaps_mean/011_S_4893.FDC.nii.gz
/Users/emmatosato/Documents/PhD/General/Verona/FCmaps_mean/011_S_6303.FDC.nii.gz
/Users/emmatosato/Documents/

Threshold and mask

In [ ]:
# Set all values below 0.2 to 0
df = df.mask(df < 0.2, 0)

# Dimensionality Reduction with UMAP